In [20]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import numpy.linalg as la
import pandas as pd
from scipy.linalg import svd
from mpl_toolkits.mplot3d import Axes3D
import random
from pipeline import generate_basic_exploration

In [109]:
df = pd.read_csv('allegations_cleaned2.csv')
df = df.drop(columns='board_disposition')

In [87]:
odf = pd.read_csv('officer_df.csv')

In [33]:
officer_exclude_cols =  ['substantiated',
                         'ever_charged']

In [5]:
len(odf)

3996

In [7]:
np.mean(odf['ever_charged'])

0.43743743743743746

In [64]:
np.mean(df['closure_time'])

296.2687211463517

In [9]:
np.mean(odf['demotion']), np.mean(odf['promotion'])

(0.017767767767767766, 0.5147647647647647)

In [42]:
# helper functions
def get_svd(trainX,regularization_param,param=0, verbose=False):
    # Calculate truncated SVD
    if verbose:
        print('Generating SVD')
    U, S, V = la.svd(trainX, full_matrices=False)
#     print('Initial U: {}'.format(str(U[:5])))
#     print('Initial S: {}'.format(str(S[:5])))
#     print('Initial V: {}'.format(str(V[:5])))
    
    if regularization_param =='trunc':
        # take inverse of each elemant
        S_inv = np.reciprocal(S)
#         print('First values of inverse: {}'.format(str(S_inv[:5])))
        S_inv[param:] = 0
    if regularization_param == 'rls':
        S_inv = S/(np.square(S)+param)

    new_S = np.zeros((U.T.shape[0],V.shape[0])) 
    np.fill_diagonal(new_S,S_inv)
    if verbose:
        print(V.T.shape, new_S.T.shape, U.T.shape)
    w_hat = V.T@new_S.T@U.T

    return w_hat
    
def key_min_val(d):
#      a) create a list of the dict's keys and values; 
#      b) return the key with the min value
    v=list(d.values())
    k=list(d.keys())
    
    return k[v.index(min(v))]


def rmse(y_hat, y):
    return np.sqrt(((y_hat - y) ** 2).mean())

def mse(y_hat, y):
    return ((y_hat - y) ** 2).mean()



def get_error_rate(w_hat,testX,testY,outcome_var_type):
#     print('testX shape is {}, w_hat shape is {}'.format(testX.shape, w_hat.shape))
    y_hat = testX@w_hat
#     print(y_hat[:20])
    
    if outcome_var_type == "binary":
        y_hat_encode = [0 if x <= 0.5
                        else 1 
                        for x in y_hat]
        
    if outcome_var_type == 'three-class':    
        y_hat_encode = [-1 if x <= -0.5 
                      else 0 if x > -0.5 and x < 0.5 
                      else 1
                     for x in y_hat]

    if outcome_var_type not in ("binary", "three-class"):
        return rmse(y_hat,testY)

    else:
        equal = np.sum(np.array(y_hat_encode).reshape(len(testY),) == np.array(testY).reshape(len(testY),))
#         print(y_hat_encode[:5], testY[:5])
#         print('sum y_hat_encode is {}'.format(sum(y_hat_encode)), 
#               'len y_hat_encode is {}'.format(len(y_hat_encode)))
#         print('mean equal is {}'.format(equal.mean()))
#         print('len testY = {}'.format(len(testY)))
        error_rate = (len(testY)-equal)/len(testY)

        return error_rate
    

In [62]:
def get_errors(df,regularization_param,outcome_var,outcome_var_type="binary",
               num_trials=10, officer_df=False, vb=False):
    outer_error_rates = list()

    outcomeList = ['outcome','demotion', 'closure_time',
                   'promotion', 'officer_charged']
    if officer_df:
        outcomeList = ['substantiated', 'ever_charged', 'demotion',
                       'officer_charged', 'promotion']
    randomState = 2
    for i in range(num_trials):
        
        inner_error_rates = dict()
        
        train,test = train_test_split(df,randomState=randomState, verbose=vb)
        trainY = np.array(train[[outcome_var]])
        trainX = np.array(train.drop(['officer_id','Unnamed: 0']+outcomeList,axis=1))
        testY = np.array(test[[outcome_var]])
        testX = np.array(test.drop(['officer_id','Unnamed: 0']+outcomeList,axis=1))
        
        randomState += 1

        if regularization_param == "trunc":
            for k in range(1,10):
                w_hat = get_svd(trainX,"trunc",param=k, verbose=vb)@trainY
                inner_error_rates[k] = get_error_rate(w_hat,testX,testY,outcome_var_type)

        if regularization_param == "rls":
            for lambda_ in np. array ([0 , 0.5 , 1, 2, 4, 8, 16]):
                w_hat = get_svd(trainX,"rls",param=lambda_, verbose=vb)@trainY
                inner_error_rates[lambda_] = get_error_rate(w_hat,testX,testY,outcome_var_type)

        min_key = key_min_val(inner_error_rates)
        outer_error_rates.append((min_key,inner_error_rates[min_key]))
            
    return outer_error_rates


In [39]:
# Group train/test by officer ID somehow? See Piazza post
def train_test_split(df,holdOut=0.2, randomState = 1, verbose=False):
    # Train, val, test split:
    # get number of unique ids and the uniqe IDs
    n_ID = len(df.officer_id.unique())
    ids = pd.DataFrame(df.officer_id.unique())

    # sample from IDs
    train_index = ids.sample(round(n_ID*(1-holdOut)),random_state = randomState ).values.tolist()
    #train_index = [item for sublist in train_index for item in sublist]
    train_index = [x[0] for x in train_index]
    # train data is data from any IDs that show up in train index
    train_data = df[df.officer_id.isin(train_index)]
    # test data is data from any IDs that don't show up in train index
    test_data = df[~df.officer_id.isin(train_index)]

    # Sanity check
    if verbose:
        print("Total Number of Unique IDs:" , len(df.officer_id.unique()))
        print("Total Number of IDs in Test Data:" , len(test_data.officer_id.unique()))
        print("Total Number of IDs in Train Data:" , len(train_data.officer_id.unique()))
        print("Do the IDs add up?" , len(test_data.officer_id.unique()) + len(train_data.officer_id.unique())  ==  len(df.officer_id.unique()))
        print("Does Test Represent 20% of the data?", (len(test_data.officer_id.unique())/len(df.officer_id.unique())) == holdOut)
        print("Test Represents X% of the data:", (len(test_data.officer_id.unique())/len(df.officer_id.unique())))
    
    return train_data, test_data

In [100]:
ridge_closure = get_errors(df,regularization_param="rls",outcome_var="closure_time",
                   outcome_var_type="time", num_trials = 50)
trunc_closure = get_errors(df,regularization_param="trunc",outcome_var="closure_time",
                   outcome_var_type="time", num_trials = 50)

In [114]:
ridge_closure, trunc_closure, np.mean(df['closure_time'])

([(2.0, 131.5778457861212),
  (1.0, 137.61129946435318),
  (16.0, 130.97976856682877),
  (0.5, 128.5108532952614),
  (0.5, 134.66732753731944),
  (4.0, 128.6932359778302),
  (0.0, 132.1113666375659),
  (0.5, 128.26330213493864),
  (1.0, 131.07086145972542),
  (0.5, 137.38991856832368),
  (1.0, 126.28512647449068),
  (2.0, 131.03668669124633),
  (4.0, 131.5679792871582),
  (1.0, 134.3134940733905),
  (2.0, 131.57429322910838),
  (2.0, 133.3286714859839),
  (8.0, 131.37191130253296),
  (0.5, 133.55658363630533),
  (8.0, 133.04755557176435),
  (0.5, 136.17838828072786),
  (2.0, 134.7637274416195),
  (2.0, 141.01045374099118),
  (0.5, 128.8633145886162),
  (0.5, 127.82013786229864),
  (4.0, 131.278093982553),
  (0.5, 140.32259959041752),
  (2.0, 133.21514739390193),
  (2.0, 129.31571322381788),
  (1.0, 132.5964822890891),
  (1.0, 138.42038257990063),
  (1.0, 127.88026193911332),
  (1.0, 139.54020346063055),
  (4.0, 138.0503604340281),
  (2.0, 135.4019130582826),
  (0.5, 137.99602987904962)

In [122]:
trunc_outcome = get_errors(df,regularization_param="trunc",outcome_var="outcome",
                   outcome_var_type="three-class", num_trials = 50)
ridge_outcome = get_errors(df,regularization_param="rls",outcome_var="outcome",
                   outcome_var_type="three-class", num_trials = 50)

In [123]:
trunc_outcome, ridge_outcome

([(9, 0.6990720913633119),
  (9, 0.7073674885575078),
  (9, 0.7173847544441097),
  (9, 0.6967836257309942),
  (9, 0.7112492770387507),
  (9, 0.7009276984243852),
  (1, 0.706058339566193),
  (1, 0.7163298421596106),
  (9, 0.7052797478329393),
  (1, 0.7110294117647059),
  (9, 0.7131172372008482),
  (9, 0.7120633988846492),
  (9, 0.7120335551055829),
  (9, 0.7115706249049719),
  (9, 0.7103068450039339),
  (9, 0.7072393382910912),
  (9, 0.7134283570892723),
  (1, 0.7076137558191921),
  (9, 0.7084860032243882),
  (7, 0.7184170471841704),
  (9, 0.7134591961023142),
  (9, 0.7153779572994806),
  (7, 0.7081420263238445),
  (1, 0.7232966379053853),
  (1, 0.7153207547169811),
  (9, 0.7007610350076103),
  (7, 0.6944735659727422),
  (3, 0.7037993920972644),
  (9, 0.725),
  (9, 0.7163586375439133),
  (1, 0.7037146226415094),
  (9, 0.7106033977738723),
  (1, 0.7237407862407862),
  (9, 0.7117045091141669),
  (9, 0.7050108932461874),
  (1, 0.7149300155520996),
  (9, 0.6943516550258123),
  (1, 0.6996413

In [102]:
o_ridge_ever_charged = get_errors(odf,regularization_param='rls',outcome_var='ever_charged',
                                  outcome_var_type="binary", num_trials=50, 
                                  officer_df=True, vb=False)
o_trunc_ever_charged = get_errors(odf,regularization_param='trunc',outcome_var='ever_charged',
                                  outcome_var_type="binary", num_trials=50, 
                                  officer_df=True, vb=False)

In [117]:
o_ridge_ever_charged, o_trunc_ever_charged, np.mean(odf['ever_charged'])

([(1.0, 0.3341677096370463),
  (2.0, 0.36795994993742176),
  (0.0, 0.3379224030037547),
  (16.0, 0.37296620775969963),
  (4.0, 0.327909887359199),
  (4.0, 0.344180225281602),
  (16.0, 0.3541927409261577),
  (8.0, 0.37922403003754696),
  (1.0, 0.3579474342928661),
  (8.0, 0.3642052565707134),
  (4.0, 0.35294117647058826),
  (8.0, 0.344180225281602),
  (0.5, 0.34167709637046306),
  (16.0, 0.35168961201501875),
  (16.0, 0.3742177722152691),
  (0.0, 0.36295369211514394),
  (16.0, 0.360450563204005),
  (16.0, 0.3316645807259074),
  (1.0, 0.360450563204005),
  (16.0, 0.36295369211514394),
  (0.0, 0.32665832290362956),
  (16.0, 0.36545682102628285),
  (4.0, 0.3617021276595745),
  (0.5, 0.3579474342928661),
  (2.0, 0.35544430538172717),
  (0.5, 0.32540675844806005),
  (16.0, 0.3504380475594493),
  (16.0, 0.3617021276595745),
  (8.0, 0.3579474342928661),
  (4.0, 0.3742177722152691),
  (0.0, 0.3591989987484355),
  (2.0, 0.34668335419274093),
  (4.0, 0.34918648310387984),
  (16.0, 0.3266583229036

In [103]:
o_ridge_promoted = get_errors(odf,regularization_param='rls',outcome_var='promotion',
                                  outcome_var_type="binary", num_trials=50, 
                                  officer_df=True, vb=False)
o_trunc_promoted = get_errors(odf,regularization_param='trunc',outcome_var='promotion',
                                  outcome_var_type="binary", num_trials=50, 
                                  officer_df=True, vb=False)

In [118]:
o_trunc_promoted, o_ridge_promoted, np.mean(odf['promotion'])

([(9, 0.3742177722152691),
  (9, 0.3667083854818523),
  (8, 0.35544430538172717),
  (9, 0.37546933667083854),
  (9, 0.376720901126408),
  (9, 0.34668335419274093),
  (9, 0.3504380475594493),
  (9, 0.3717146433041302),
  (9, 0.33917396745932415),
  (9, 0.327909887359199),
  (9, 0.3642052565707134),
  (9, 0.35544430538172717),
  (9, 0.3667083854818523),
  (9, 0.35544430538172717),
  (9, 0.36545682102628285),
  (8, 0.3504380475594493),
  (8, 0.3742177722152691),
  (9, 0.34167709637046306),
  (9, 0.3692115143929912),
  (9, 0.3566958698372966),
  (6, 0.36795994993742176),
  (9, 0.344180225281602),
  (9, 0.37797246558197745),
  (9, 0.38172715894868586),
  (9, 0.3642052565707134),
  (9, 0.34918648310387984),
  (6, 0.3804755944931164),
  (9, 0.3379224030037547),
  (9, 0.37546933667083854),
  (9, 0.36295369211514394),
  (8, 0.3704630788485607),
  (9, 0.35168961201501875),
  (9, 0.3642052565707134),
  (8, 0.38172715894868586),
  (9, 0.3579474342928661),
  (9, 0.37296620775969963),
  (9, 0.381727

In [104]:
o_ridge_demoted = get_errors(odf,regularization_param='rls',outcome_var='demotion',
                                  outcome_var_type="binary", num_trials=50, 
                                  officer_df=True, vb=False)
o_trunc_demoted = get_errors(odf,regularization_param='trunc',outcome_var='demotion',
                                  outcome_var_type="binary", num_trials=50, 
                                  officer_df=True, vb=False)

In [119]:
o_trunc_demoted, o_ridge_demoted, np.mean(odf['demotion'])

([(1, 0.023779724655819776),
  (1, 0.017521902377972465),
  (1, 0.017521902377972465),
  (1, 0.02002503128911139),
  (1, 0.017521902377972465),
  (1, 0.016270337922403004),
  (1, 0.015018773466833541),
  (1, 0.01877346683354193),
  (1, 0.025031289111389236),
  (1, 0.02002503128911139),
  (1, 0.015018773466833541),
  (1, 0.015018773466833541),
  (1, 0.016270337922403004),
  (1, 0.02127659574468085),
  (1, 0.016270337922403004),
  (1, 0.016270337922403004),
  (1, 0.012515644555694618),
  (1, 0.011264080100125156),
  (1, 0.01376720901126408),
  (1, 0.02002503128911139),
  (1, 0.011264080100125156),
  (1, 0.017521902377972465),
  (1, 0.01877346683354193),
  (1, 0.015018773466833541),
  (1, 0.025031289111389236),
  (1, 0.01376720901126408),
  (1, 0.023779724655819776),
  (1, 0.02127659574468085),
  (1, 0.016270337922403004),
  (1, 0.02127659574468085),
  (1, 0.015018773466833541),
  (1, 0.01877346683354193),
  (1, 0.017521902377972465),
  (1, 0.02252816020025031),
  (1, 0.02127659574468085)

In [105]:
o_trunc_substantiated = get_errors(odf,regularization_param='trunc',outcome_var='substantiated',
                                  outcome_var_type="continuous", num_trials=50, 
                                  officer_df=True, vb=False)
o_ridge_substantiated = get_errors(odf,regularization_param='rls',outcome_var='substantiated',
                                  outcome_var_type="continuous", num_trials=50, 
                                  officer_df=True, vb=False)

In [120]:
o_trunc_substantiated, o_ridge_substantiated, np.mean(odf['substantiated'])

([(9, 1.4445545601389942),
  (9, 1.4662648699605993),
  (5, 1.3666337344841948),
  (9, 1.4429534947391456),
  (7, 1.5661161972939974),
  (7, 1.4166632670068346),
  (9, 1.534965727536938),
  (9, 1.4478032972957566),
  (9, 1.3986373487005446),
  (9, 1.5154031254523574),
  (9, 1.2399723368838889),
  (9, 1.3493200379010535),
  (9, 1.4689592301511711),
  (9, 1.5684799607970166),
  (5, 1.2532617301171565),
  (9, 1.4338740093684146),
  (9, 1.4507500623793055),
  (9, 1.4360428015706106),
  (9, 1.3993205579616614),
  (9, 1.355302599993845),
  (9, 1.389724525932441),
  (9, 1.3802180634429864),
  (6, 1.3428509748328403),
  (5, 1.3706177514742268),
  (6, 1.2922298510525936),
  (9, 1.489813656685988),
  (5, 1.4591231988850863),
  (9, 1.370199602145142),
  (7, 1.3766398579333754),
  (9, 1.432333004433521),
  (5, 1.507679274972666),
  (9, 1.408887889906318),
  (8, 1.4053319751615532),
  (5, 1.374146642609442),
  (9, 1.3761223402025182),
  (9, 1.3994290946552235),
  (5, 1.4857176005748922),
  (9, 1.42

In [106]:
o_trunc_charged = get_errors(odf,regularization_param='trunc',outcome_var='officer_charged',
                                  outcome_var_type="continuous", num_trials=50, 
                                  officer_df=True, vb=False)
o_ridge_charged = get_errors(odf,regularization_param='rls',outcome_var='officer_charged',
                                  outcome_var_type="continuous", num_trials=50, 
                                  officer_df=True, vb=False)

In [121]:
o_trunc_charged, o_ridge_charged, np.mean(odf['officer_charged'])

([(9, 1.3335991451183489),
  (6, 1.5927084851564315),
  (5, 1.4137456994490714),
  (9, 1.4405635032738957),
  (9, 1.4892897458071737),
  (7, 1.4418533842150576),
  (7, 1.531970643381867),
  (9, 1.5657789701602123),
  (8, 1.4128095429212708),
  (9, 1.507112583159253),
  (8, 1.313037191318337),
  (7, 1.303791304493412),
  (8, 1.366971536447982),
  (5, 1.552402031438257),
  (6, 1.318993499710545),
  (8, 1.5134664749089148),
  (8, 1.3457327677833668),
  (8, 1.4901228089394116),
  (5, 1.3998612697298671),
  (7, 1.4398983242214),
  (8, 1.3254374093274939),
  (9, 1.4087972671555928),
  (7, 1.4620058179175623),
  (4, 1.390552681822451),
  (9, 1.3742066372420494),
  (8, 1.444924424798773),
  (8, 1.6296281832152197),
  (5, 1.4214381926020025),
  (9, 1.4349001425017556),
  (8, 1.392052469199597),
  (9, 1.5330958550946359),
  (5, 1.447010249686909),
  (8, 1.5236188904849537),
  (8, 1.3814652902865985),
  (8, 1.288096337723136),
  (9, 1.3663868429528903),
  (5, 1.5470044491684618),
  (8, 1.41686677